数据分布算法根据时间由旧到新经历了不同发展, 以下按照时间顺序进行罗列

#### 一. Hash算法
1. 算法:  
   用mod取余, 计算数据发往哪台master : key mod $n_{master}$  
  
2. 弊端  
   假设有3台master, 如果其中有1台宕机了, 则$n_{master}$ 变小, 此后从缓存取数据时再用新的$n_{master}$ mod取余可能使得大部分数据都取不到, 导致缓存不可用  
   ```eg: 原来3台机器, key=8, (8 mod 3)=2; 1台机器宕机后变成2台机器, 取key=8, (8 mod 2)=0```
   
#### 二. 一致性Hash算法
1. 算法目标  
   解决分布式存储架构下, 动态增加或删除节点带来的问题
   
2. 算法内容
    1. 首先, 把全量的缓存空间设置为一个环形结构, 并均匀的分成2^32个缓冲区.   
       分发数据时, hash取余的底数设置为固定的2^32, 让数据均匀分布
    2. 然后, 将节点也按照相同的hash取余算法放置到环形缓冲区中(可以用节点的ip作为hashcode)
    3. 如何将数据和节点对应起来呢? 让圆环中的数据按照顺时针找到的第一个节点就是该数据所属的节点  
    <img src="img/chash2.png">
    4. 有的时候, 几个节点根据ip计算hash取余后, 在环形缓冲区中可能爱得很近, 导致大量的key都落在一个节点上.  
       为了解决这个问题, 引入"虚拟节点": 就是基于原来每个物理节点, 映射出N个虚拟节点  
        * 如下图, 假如node1的ip是192.168.1.109，那么原node1节点在环形空间的位置就是hash（“192.168.1.109”）。  
        我们基于node1构建两个虚拟节点，node1-1 和 node1-2，虚拟节点在环形空间的位置可以利用（IP+后缀）计算，  
        例如：hash(“192.168.1.109#1”)，hash(“192.168.1.109#2”)
       
       此时，环形空间中不再有物理节点node1，node2，只有虚拟节点node1-1，node1-2，node2-1，node2-2。  
       由于虚拟节点数量较多，缓存key与虚拟节点的映射关系也变得相对均衡了。
        <img src="img/chash1.png" width="50%">
        
3. 一致性哈希算法的改进效果   
    * 普通哈希算法, 在节点增加/宕机后, 接近100%的key都可能无法查到;   
    * 而一致性哈希, 只有落在"新增/删除节点"与其上一个节点之间的key无法找到, 使受影响的key降低为$\frac{1}{n}$
    
4. [一致性hash的简单实现](ConsistentHash.java)
<img src="img/hash-code.png">
    
#### 三. redis的做法
1. 算法内容  
    1. redis将hash取余的底数设置为固定的16384, 代表数据会被分发到16384个不同的slot中;   
    2. 而每台机器会均匀的持有一定数量的slot, 比如集群中有3个节点 : 
        * 节点1: 持有0~5000号的slot
        * 节点2: 持有5001~10000的slot
        * 节点3: 持10001~16384的slot  当redis分发一个key时, 
 
 当某台机器宕机后, 因为查找key时, 取余的底数还是固定的16384, 所以hash结果和插入key时的hash结果一致, 并不会影响全部数据, 只是宕机的那个节点上的slot数据取不到; 当redis发现某个节点宕机后, 就会将原本属于这个节点的slot均匀的分配到其它节点 